In [20]:
# Packages

import pandas as pd
import datetime
import re
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
# Importing the Merged Train Data Sheet

train_CSV_path = '../trainData/cleanAllTrainData.csv'

train_data = pd.read_csv(train_CSV_path)

In [13]:
train_data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28946115 entries, 0 to 28946114
Data columns (total 14 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Unnamed: 0.1               int64  
 1   Unnamed: 0                 int64  
 2   Train #                    int64  
 3   Origin Date                object 
 4   Sch Dp                     object 
 5   Act Dp                     object 
 6   Comments                   object 
 7   Service Disruption         object 
 8   Cancellations              object 
 9   Station Code               object 
 10  Weekday                    object 
 11  Date Time                  object 
 12  Delay                      int64  
 13  day_shifted_average_delay  float64
dtypes: float64(1), int64(4), object(9)
memory usage: 14.3 GB


In [14]:
train_data.head(8)

,Unnamed: 0.1,Unnamed: 0,Train #,Origin Date,Sch Dp,Act Dp,Comments,Service Disruption,Cancellations,Station Code,Weekday,Date Time,Delay,day_shifted_average_delay
0,0,0,140,12/31/2022 (Sa),12/31/2022 6:40 PM (Sa),6:45PM,Dp: 5 min late.,NaN,NaN,WFD,Sa,2022-12-31 18:40:00,5,7.908732
1,1,1,467,12/31/2022 (Sa),12/31/2022 6:32 PM (Sa),6:33PM,Dp: 1 min late.,NaN,NaN,WFD,Sa,2022-12-31 18:32:00,1,7.908732
2,2,2,488,12/31/2022 (Sa),12/31/2022 5:14 PM (Sa),5:26PM,Dp: 12 min late.,NaN,NaN,WFD,Sa,2022-12-31 17:14:00,12,7.908732
3,3,3,464,12/31/2022 (Sa),12/31/2022 3:14 PM (Sa),3:35PM,Dp: 21 min late.,NaN,NaN,WFD,Sa,2022-12-31 15:14:00,21,7.908732
4,4,4,463,12/31/2022 (Sa),12/31/2022 1:32 PM (Sa),1:33PM,Dp: 1 min late.,NaN,NaN,WFD,Sa,2022-12-31 13:32:00,1,7.908732
5,5,5,461,12/31/2022 (Sa),12/31/2022 11:36 AM (Sa),11:37AM,Dp: 1 min late.,NaN,NaN,WFD,Sa,2022-12-31 11:36:00,1,7.908732
6,6,6,460,12/31/2022 (Sa),12/31/2022 11:14 AM (Sa),11:15AM,Dp: 1 min late.,NaN,NaN,WFD,Sa,2022-12-31 11:14:00,1,7.908732
7,7,7,147,12/31/2022 (Sa),12/31/2022 9:41 AM (Sa),9:46AM,Dp: 5 min late.,NaN,NaN,WFD,Sa,2022-12-31 09:41:00,5,7.908732


In [15]:
train_data.describe()

,Unnamed: 0.1,Unnamed: 0,Train #,Delay,day_shifted_average_delay
count,2.894612e+07,2.894612e+07,2.894612e+07,2.894612e+07,2.894612e+07
mean,1.447401e+07,7.279747e+03,4.378551e+02,1.096482e+01,1.094718e+01
std,8.356582e+06,9.391755e+03,5.056963e+02,2.872914e+01,5.632947e+00
min,0.000000e+00,0.000000e+00,1.000000e+00,-3.600000e+02,2.665186e+00
25%,7.237018e+06,1.227000e+03,8.700000e+01,0.000000e+00,7.468382e+00
50%,1.447394e+07,3.770000e+03,3.060000e+02,3.000000e+00,8.803653e+00
75%,2.171106e+07,9.224000e+03,6.460000e+02,1.100000e+01,1.261824e+01
max,2.894799e+07,5.807900e+04,9.998000e+03,2.160000e+03,4.986985e+01


In [ ]:
# Optimizing the Data Types to Reduce Memory Usage

# Dropping old ID columns
train_data = train_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'day_shifted_average_delay', 'Date Time'])

# Downcast integers
train_data['Train #'] = train_data['Train #'].astype('int32')
train_data['Delay'] = train_data['Delay'].astype('int16')

# Categorical columns
categorical_columns = ['Station Code', 'Weekday', 'Cancellations']
for col in categorical_columns:
    train_data[col] = train_data[col].astype('category')

# Converting datetime columns

# Origin Date - extract just the date part
train_data['Origin Date'] = pd.to_datetime(train_data['Origin Date'].str.split(' ').str[0], format='%m/%d/%Y')

# Sch Dp - full datetime with date and time
train_data['Sch Dp'] = pd.to_datetime(train_data['Sch Dp'].str.rsplit(' ', n=1).str[0], format='%m/%d/%Y %I:%M %p')

# Act Dp - combine date from Sch Dp and time from Act Dp
train_data['Act Dp'] = pd.to_datetime(
    train_data['Sch Dp'].dt.date.astype(str) + ' ' + train_data['Act Dp'].str.rsplit(' ', n=1).str[0],
    format='%Y-%m-%d %I:%M%p'
)

ValueError: time data "2017-05-22 7:91AM" doesn't match format "%Y-%m-%d %I:%M%p", at position 5220106. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [21]:
# Extract just the time portion (rightmost part of string)
act_times = train_data['Act Dp'].str.rsplit(' ', n=1).str[0]

# Regex for a valid 12-hour time like "7:05AM", "12:59PM", etc.
valid_time_pattern = re.compile(r'^(0?[1-9]|1[0-2]):[0-5][0-9][AP]M$')

# Create a boolean mask for invalid times
invalid_mask = ~act_times.str.match(valid_time_pattern, na=False)

# Get the rows with invalid times
invalid_rows = train_data[invalid_mask].copy()

# Show how many and a few examples
print(f"Found {invalid_rows.shape[0]} invalid times in 'Act Dp'.")
print(invalid_rows[['Act Dp']].head(10))

Found 1787605 invalid times in 'Act Dp'.
     Act Dp
253     NaN
454     NaN
516     NaN
537     NaN
552     NaN
553     NaN
587     NaN
1025    NaN
1189    NaN
1287    NaN


In [25]:
print(len(train_data['Act Dp']))
print(train_data['Act Dp'][253])


28946115
nan


In [29]:
print(train_data.iloc[253])
print(train_data.iloc[454])

Train #                               451
Origin Date           2022-12-14 00:00:00
Sch Dp                2022-12-14 06:10:00
Act Dp                                NaN
Comments                              NaN
Service Disruption                     SD
Cancellations                         NaN
Station Code                          WFD
Weekday                                We
Delay                                   0
Name: 253, dtype: object
Train #                               490
Origin Date           2022-11-30 00:00:00
Sch Dp                2022-11-30 09:19:00
Act Dp                                NaN
Comments                              NaN
Service Disruption                    NaN
Cancellations                         NaN
Station Code                          WFD
Weekday                                We
Delay                                   0
Name: 454, dtype: object


In [ ]:
train_data.head(8)

,Train #,Origin Date,Sch Dp,Act Dp,Comments,Service Disruption,Cancellations,Station Code,Weekday,Delay
0,140,2022-12-31,2022-12-31 18:40:00,6:45PM,Dp: 5 min late.,NaN,NaN,WFD,Sa,5
1,467,2022-12-31,2022-12-31 18:32:00,6:33PM,Dp: 1 min late.,NaN,NaN,WFD,Sa,1
2,488,2022-12-31,2022-12-31 17:14:00,5:26PM,Dp: 12 min late.,NaN,NaN,WFD,Sa,12
3,464,2022-12-31,2022-12-31 15:14:00,3:35PM,Dp: 21 min late.,NaN,NaN,WFD,Sa,21
4,463,2022-12-31,2022-12-31 13:32:00,1:33PM,Dp: 1 min late.,NaN,NaN,WFD,Sa,1
5,461,2022-12-31,2022-12-31 11:36:00,11:37AM,Dp: 1 min late.,NaN,NaN,WFD,Sa,1
6,460,2022-12-31,2022-12-31 11:14:00,11:15AM,Dp: 1 min late.,NaN,NaN,WFD,Sa,1
7,147,2022-12-31,2022-12-31 09:41:00,9:46AM,Dp: 5 min late.,NaN,NaN,WFD,Sa,5


In [ ]:
train_data.describe()

,Train #,Origin Date,Sch Dp,Delay
count,2.894612e+07,28946115,28946115,2.894612e+07
mean,4.378551e+02,2016-06-19 07:49:05.101583616,2016-06-20 01:17:15.262543104,1.096482e+01
min,1.000000e+00,2009-12-31 00:00:00,2010-01-02 00:07:00,-3.600000e+02
25%,8.700000e+01,2013-08-14 00:00:00,2013-08-14 12:09:00,0.000000e+00
50%,3.060000e+02,2016-06-15 00:00:00,2016-06-15 12:34:00,3.000000e+00
75%,6.460000e+02,2019-03-20 00:00:00,2019-03-20 07:28:00,1.100000e+01
max,9.998000e+03,2022-12-31 00:00:00,2022-12-31 23:59:00,2.160000e+03
std,5.056963e+02,NaN,NaN,2.872914e+01


In [ ]:
# Checking for wiered delay cases over midnight
mask = df['Act Dp'] < df['Sch Dp']

In [ ]:
train_data.to_parquet('../trainData/optimized_train_data.parquet', compression='snappy')

train_data.memory_usage(deep=True)